# バックテスト・仮想ポートフォリオ サンプルノートブック

このノートブックでは、バックテスト機能と仮想ポートフォリオ機能を使った投資戦略の検証ワークフローを学びます。

## 使用するクラス

| クラス | 主な用途 |
|--------|----------|
| **Backtester** | シグナルベースのバックテスト実行 |
| **BacktestResults** | バックテスト結果の分析・可視化・出力 |
| **VirtualPortfolio** | 仮想ポートフォリオの作成・追跡 |

## 前提条件
- J-Quants APIでデータ収集済み（`data/jquants.db`が存在）
- 分析スクリプト実行済み（`data/analysis_results.db`が存在）

In [ ]:
# 基本インポート
import pandas as pd

# Bokeh警告を抑制（backtesting.pyが内部で使用）
import backtesting

backtesting.set_bokeh_output(notebook=False)

# バックテスト関連クラスをインポート
from technical_tools import (  # noqa: E402
    Backtester,
    VirtualPortfolio,
    StockScreener,
    ScreenerFilter,
    TechnicalAnalyzer,
)

# Market Readerもインポート

print("インポート完了")

---
## 2. Backtester: バックテストの基本

`Backtester`クラスはシグナルベースのバックテストエンジンです。
シグナル追加 → 退出ルール追加 → バックテスト実行 の流れで使用します。

### 初期化パラメータ
| パラメータ | 説明 | デフォルト |
|------------|------|------------|
| `cash` | 初期資金 | 1,000,000円 |
| `commission` | 手数料率 | 0（無料） |

In [2]:
# Backtesterの初期化
bt = Backtester(cash=1_000_000, commission=0.001)  # 0.1%の手数料
print(f"Backtester初期化: {bt}")

Backtester初期化: Backtester(signals=[], cash=1000000)


### 対応シグナル一覧

| シグナル名 | 説明 | 主なパラメータ |
|------------|------|----------------|
| `golden_cross` | 短期MAが長期MAを上抜け | `short`, `long` |
| `dead_cross` | 短期MAが長期MAを下抜け | `short`, `long` |
| `rsi_oversold` | RSIが閾値以下（売られすぎ） | `threshold`, `period` |
| `rsi_overbought` | RSIが閾値以上（買われすぎ） | `threshold`, `period` |
| `macd_cross` | MACDがシグナル線を上抜け | `fast`, `slow`, `signal_period` |
| `bollinger_breakout` | 価格がBBを突破 | `period`, `std_dev`, `direction` |
| `bollinger_squeeze` | BBスクイーズ後の拡大 | `period`, `std_dev` |
| `volume_spike` | 出来高急増 | `threshold`, `period` |
| `volume_breakout` | 高値更新+出来高増 | `volume_threshold`, `price_period` |

In [3]:
# シグナルの追加（メソッドチェーン可能）
bt = Backtester(cash=1_000_000)
bt.add_signal("golden_cross", short=5, long=25)
print(f"シグナル追加後: {bt}")

シグナル追加後: Backtester(signals=['golden_cross'], cash=1000000)


### 退出ルール一覧

| ルール名 | 説明 | パラメータ例 |
|----------|------|--------------|
| `stop_loss` | 損切り（負の値） | `threshold=-0.10` (10%下落で損切り) |
| `take_profit` | 利確（正の値） | `threshold=0.20` (20%上昇で利確) |
| `max_holding_days` | 最大保有日数 | `days=30` |
| `trailing_stop` | トレーリングストップ | `threshold=-0.05` |

In [4]:
# 退出ルールの追加
bt.add_exit_rule("stop_loss", threshold=-0.10)  # 10%損切り
bt.add_exit_rule("take_profit", threshold=0.20)  # 20%利確
print(f"退出ルール追加後: {bt}")

退出ルール追加後: Backtester(signals=['golden_cross'], cash=1000000)


In [5]:
# バックテストの実行
# 注: 実際のデータベースが必要です
try:
    results = bt.run(
        symbols=["7203"],  # トヨタ
        start="2024-01-01",
        end="2024-12-31",
    )
    print(f"バックテスト完了: {results}")
except Exception as e:
    print(f"エラー: {e}")
    print("デモ用にダミー結果を使用します")
    results = None

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

バックテスト完了: BacktestResults(trades=4, win_rate=25.0%, profit_factor=0.40)


In [6]:
# 結果サマリーの確認
if results:
    summary = results.summary()
    print("=== バックテスト結果サマリー ===")
    for key, value in summary.items():
        if isinstance(value, float) and (
            "rate" in key or "return" in key or "drawdown" in key
        ):
            print(f"  {key}: {value:.2%}")
        else:
            print(f"  {key}: {value}")

=== バックテスト結果サマリー ===
  total_trades: 4
  win_rate: 25.00%
  avg_return: -0.04%
  max_return: 0.16%
  max_loss: -0.0013
  profit_factor: 0.4
  max_drawdown: 35.00%
  sharpe_ratio: -0.77
  avg_holding_days: 47.0


---
## 3. シグナル詳細: 各シグナルの使い方

各シグナルの設定方法と組み合わせを詳しく見ていきます。

In [7]:
# ゴールデンクロス/デッドクロス
# 短期移動平均線が長期を上抜け（買い）/ 下抜け（売り）

bt_gc = Backtester()
bt_gc.add_signal("golden_cross", short=5, long=25)
bt_gc.add_exit_rule("stop_loss", threshold=-0.10)
bt_gc.add_exit_rule("take_profit", threshold=0.15)

print("ゴールデンクロス戦略:")
print("  シグナル: 5日線が25日線を上抜け")
print("  損切り: -10%")
print("  利確: +15%")

ゴールデンクロス戦略:
  シグナル: 5日線が25日線を上抜け
  損切り: -10%
  利確: +15%


In [8]:
# RSIシグナル（売られすぎ/買われすぎ）
# RSIが閾値を下回ったら買い、上回ったら売りを示唆

bt_rsi = Backtester()
bt_rsi.add_signal("rsi_oversold", threshold=30, period=14)  # RSI < 30で買い
bt_rsi.add_exit_rule("stop_loss", threshold=-0.08)
bt_rsi.add_exit_rule("take_profit", threshold=0.12)

print("RSI売られすぎ戦略:")
print("  シグナル: RSI(14) < 30")
print("  損切り: -8%")
print("  利確: +12%")

RSI売られすぎ戦略:
  シグナル: RSI(14) < 30
  損切り: -8%
  利確: +12%


In [9]:
# MACDクロスシグナル
# MACDラインがシグナルラインを上抜け

bt_macd = Backtester()
bt_macd.add_signal("macd_cross", fast=12, slow=26, signal_period=9)
bt_macd.add_exit_rule("stop_loss", threshold=-0.10)
bt_macd.add_exit_rule("max_holding_days", days=30)

print("MACDクロス戦略:")
print("  シグナル: MACD(12/26/9)がシグナルを上抜け")
print("  損切り: -10%")
print("  最大保有: 30日")

MACDクロス戦略:
  シグナル: MACD(12/26/9)がシグナルを上抜け
  損切り: -10%
  最大保有: 30日


In [10]:
# ボリンジャーバンドブレイクアウト
# 価格がBBの上限/下限を突破

bt_bb = Backtester()
bt_bb.add_signal("bollinger_breakout", period=20, std_dev=2.0, direction="up")
bt_bb.add_exit_rule("trailing_stop", threshold=-0.05)

print("BBブレイクアウト戦略:")
print("  シグナル: 価格がBB上限(2σ)を突破")
print("  トレーリングストップ: -5%")

BBブレイクアウト戦略:
  シグナル: 価格がBB上限(2σ)を突破
  トレーリングストップ: -5%


In [11]:
# ボリンジャースクイーズ
# バンド収縮後の拡大を狙う

bt_squeeze = Backtester()
bt_squeeze.add_signal("bollinger_squeeze", period=20, std_dev=2.0)
bt_squeeze.add_exit_rule("take_profit", threshold=0.15)
bt_squeeze.add_exit_rule("stop_loss", threshold=-0.07)

print("BBスクイーズ戦略:")
print("  シグナル: BBが収縮後に拡大")
print("  利確: +15%, 損切り: -7%")

BBスクイーズ戦略:
  シグナル: BBが収縮後に拡大
  利確: +15%, 損切り: -7%


In [12]:
# 出来高スパイク
# 出来高が移動平均のN倍を超えた

bt_vol = Backtester()
bt_vol.add_signal("volume_spike", threshold=2.0, period=20)
bt_vol.add_exit_rule("max_holding_days", days=5)

print("出来高スパイク戦略:")
print("  シグナル: 出来高が20日平均の2倍超")
print("  最大保有: 5日（短期トレード）")

出来高スパイク戦略:
  シグナル: 出来高が20日平均の2倍超
  最大保有: 5日（短期トレード）


In [13]:
# 出来高確認付きブレイクアウト
# 高値更新+出来高増加

bt_vb = Backtester()
bt_vb.add_signal("volume_breakout", volume_threshold=1.5, price_period=20)
bt_vb.add_exit_rule("trailing_stop", threshold=-0.08)
bt_vb.add_exit_rule("take_profit", threshold=0.25)

print("出来高ブレイクアウト戦略:")
print("  シグナル: 20日高値更新 + 出来高1.5倍超")
print("  トレーリング: -8%, 利確: +25%")

出来高ブレイクアウト戦略:
  シグナル: 20日高値更新 + 出来高1.5倍超
  トレーリング: -8%, 利確: +25%


In [14]:
# 複合シグナル: 複数シグナルの組み合わせ（OR条件）

bt_combo = Backtester(cash=1_000_000)

# 複数のシグナルを追加（どれか1つが発生したらエントリー）
bt_combo.add_signal("golden_cross", short=5, long=25)
bt_combo.add_signal("rsi_oversold", threshold=30)
bt_combo.add_signal("volume_breakout", volume_threshold=1.5)

# 退出ルール
bt_combo.add_exit_rule("stop_loss", threshold=-0.10)
bt_combo.add_exit_rule("take_profit", threshold=0.20)
bt_combo.add_exit_rule("trailing_stop", threshold=-0.05)

print("複合シグナル戦略:")
print("  エントリー条件（OR）:")
print("    - ゴールデンクロス(5/25)")
print("    - RSI < 30")
print("    - 出来高ブレイクアウト")
print("  退出条件:")
print("    - 損切り: -10%")
print("    - 利確: +20%")
print("    - トレーリング: -5%")

複合シグナル戦略:
  エントリー条件（OR）:
    - ゴールデンクロス(5/25)
    - RSI < 30
    - 出来高ブレイクアウト
  退出条件:
    - 損切り: -10%
    - 利確: +20%
    - トレーリング: -5%


In [15]:
# 複合シグナルのバックテスト実行
try:
    results_combo = bt_combo.run(
        symbols=["7203", "9984", "6758"],  # 複数銘柄
        start="2024-01-01",
        end="2024-12-31",
        max_workers=4,  # 並列処理
    )
    print(f"複合シグナルバックテスト完了: {results_combo}")
except Exception as e:
    print(f"エラー: {e}")
    results_combo = None

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

複合シグナルバックテスト完了: BacktestResults(trades=27, win_rate=44.4%, profit_factor=1.13)


---
## 4. BacktestResults: 結果の分析

`BacktestResults`クラスは、バックテスト結果の詳細分析と可視化機能を提供します。

### 主なメソッド
| メソッド | 説明 |
|----------|------|
| `summary()` | パフォーマンス指標のサマリー |
| `trades()` | 全取引のDataFrame |
| `plot()` | 資産推移チャート |
| `by_symbol()` | 銘柄別パフォーマンス |
| `monthly_returns()` | 月次リターン |
| `yearly_returns()` | 年次リターン |
| `export()` | CSV/Excel/HTML出力 |

In [16]:
# summary(): パフォーマンス指標
if results:
    summary = results.summary()

    print("=== パフォーマンス指標 ===")
    print(f"総取引数: {summary['total_trades']}")
    print(f"勝率: {summary['win_rate']:.1%}")
    print(f"平均リターン: {summary['avg_return']:.2%}")
    print(f"最大リターン: {summary['max_return']:.2%}")
    print(f"最大損失: {summary['max_loss']:.2%}")
    print(f"プロフィットファクター: {summary['profit_factor']:.2f}")
    print(f"最大ドローダウン: {summary['max_drawdown']:.2%}")
    print(f"シャープレシオ: {summary['sharpe_ratio']:.2f}")
    print(f"平均保有日数: {summary['avg_holding_days']:.1f}日")

=== パフォーマンス指標 ===
総取引数: 4
勝率: 25.0%
平均リターン: -0.04%
最大リターン: 0.16%
最大損失: -0.13%
プロフィットファクター: 0.40
最大ドローダウン: 35.00%
シャープレシオ: -0.77
平均保有日数: 47.0日


In [17]:
# trades(): 全取引の一覧
if results:
    trades_df = results.trades()
    print(f"取引数: {len(trades_df)}")
    print(f"カラム: {list(trades_df.columns)}")
    display(trades_df.head(10))

取引数: 4
カラム: ['symbol', 'entry_date', 'entry_price', 'exit_date', 'exit_price', 'shares', 'pnl', 'return_pct', 'holding_days', 'exit_reason']


,symbol,entry_date,entry_price,exit_date,exit_price,shares,pnl,return_pct,holding_days,exit_reason
0,7203,2024-03-22,3829.0,2024-05-14,3350.0,261,-125019.0,-0.001251,53,stop_loss
1,7203,2024-07-01,3325.0,2024-07-29,2958.0,263,-96521.0,-0.001104,28,stop_loss
2,7203,2024-08-30,2707.0,2024-09-12,2485.0,287,-63714.0,-0.000820,13,signal_exit
3,7203,2024-09-27,2746.5,2024-12-30,3182.0,260,113230.0,0.001586,94,signal_exit


In [18]:
# plot(): 資産推移チャート
if results:
    fig = results.plot()
    fig.update_layout(title="バックテスト結果: 資産推移とドローダウン")
    fig.show()

In [19]:
# by_symbol(): 銘柄別パフォーマンス
if results_combo:
    by_symbol = results_combo.by_symbol()
    print("=== 銘柄別パフォーマンス ===")
    display(by_symbol)

=== 銘柄別パフォーマンス ===


,symbol,trades,avg_return,total_pnl,win_rate
0,6758,9,0.000140,107210.0,0.555556
1,7203,10,0.000096,45548.0,0.400000
2,9984,8,0.000036,-22753.0,0.375000


In [20]:
# monthly_returns(): 月次リターン
if results:
    monthly = results.monthly_returns()
    print("=== 月次リターン ===")
    display(monthly)

=== 月次リターン ===


,year_month,trades,avg_return,total_pnl
0,2024-05,1,-0.001251,-125019.0
1,2024-07,1,-0.001104,-96521.0
2,2024-09,1,-0.000820,-63714.0
3,2024-12,1,0.001586,113230.0


In [21]:
# yearly_returns(): 年次リターン
if results:
    yearly = results.yearly_returns()
    print("=== 年次リターン ===")
    display(yearly)

=== 年次リターン ===


,year,trades,avg_return,total_pnl,win_rate
0,2024,4,-0.000397,-172024.0,0.25


In [22]:
# export(): 結果のエクスポート
if results:
    # CSV出力
    # results.export("output/backtest_results.csv")

    # Excel出力（Summary, Trades, By Symbol, Monthly Returns シート）
    # results.export("output/backtest_report.xlsx")

    # HTML出力
    # results.export("output/backtest_report.html")

    print("エクスポート例:")
    print("  results.export('report.csv')   # CSV")
    print("  results.export('report.xlsx')  # Excel (複数シート)")
    print("  results.export('report.html')  # HTML")

エクスポート例:
  results.export('report.csv')   # CSV
  results.export('report.xlsx')  # Excel (複数シート)
  results.export('report.html')  # HTML


In [23]:
# カスタム分析: 月次損益バーチャート
import plotly.graph_objects as go

if results:
    monthly = results.monthly_returns()

    if not monthly.empty:
        colors = ["green" if pnl > 0 else "red" for pnl in monthly["total_pnl"]]

        fig = go.Figure()
        fig.add_trace(
            go.Bar(
                x=monthly["year_month"],
                y=monthly["total_pnl"],
                marker_color=colors,
                name="月次損益",
            )
        )

        fig.update_layout(
            title="月次損益バーチャート",
            xaxis_title="年月",
            yaxis_title="損益 (円)",
            height=400,
        )
        fig.show()

In [24]:
# 取引の詳細分析: 勝ちトレード vs 負けトレード
if results:
    trades_df = results.trades()

    if not trades_df.empty:
        wins = trades_df[trades_df["pnl"] > 0]
        losses = trades_df[trades_df["pnl"] <= 0]

        print("=== 勝ちトレード分析 ===")
        print(f"  件数: {len(wins)}")
        if len(wins) > 0:
            print(f"  平均リターン: {wins['return_pct'].mean():.2%}")
            print(f"  平均保有日数: {wins['holding_days'].mean():.1f}日")

        print()
        print("=== 負けトレード分析 ===")
        print(f"  件数: {len(losses)}")
        if len(losses) > 0:
            print(f"  平均リターン: {losses['return_pct'].mean():.2%}")
            print(f"  平均保有日数: {losses['holding_days'].mean():.1f}日")

=== 勝ちトレード分析 ===
  件数: 1
  平均リターン: 0.16%
  平均保有日数: 94.0日

=== 負けトレード分析 ===
  件数: 3
  平均リターン: -0.11%
  平均保有日数: 31.3日


---
## 5. スクリーナー連携バックテスト

`run_with_screener()`メソッドを使うと、スクリーナーで抽出した銘柄に対してバックテストを実行できます。

### 使い方
1. スクリーナー条件を指定
2. 条件に合致する銘柄を自動抽出
3. 抽出した銘柄群でバックテスト実行

In [25]:
# スクリーナー連携バックテストの基本
bt = Backtester(cash=1_000_000)

try:
    results_screener = bt.run_with_screener(
        screener_filter={"composite_score_min": 70, "hl_ratio_min": 80},
        start="2024-01-01",
        end="2024-12-31",
        exit_rules={"stop_loss": -0.10, "take_profit": 0.20},
    )
    print(f"スクリーナー連携バックテスト完了: {results_screener}")
except Exception as e:
    print(f"エラー: {e}")
    results_screener = None

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/232 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/242 [00:00<?, ?bar/s]

スクリーナー連携バックテスト完了: BacktestResults(trades=506, win_rate=51.8%, profit_factor=1.50)


In [26]:
# ScreenerFilterオブジェクトを使用
filter_config = ScreenerFilter(
    composite_score_min=75.0,
    hl_ratio_min=80.0,
    rsi_max=70.0,  # 過熱感なし
    market_cap_min=50_000_000_000,  # 500億円以上
    limit=20,
)

bt2 = Backtester(cash=1_000_000)

try:
    results_filter = bt2.run_with_screener(
        screener_filter=filter_config,
        start="2024-01-01",
        end="2024-12-31",
        exit_rules={"stop_loss": -0.08, "take_profit": 0.15, "max_holding_days": 30},
    )
    print(f"ScreenerFilter使用結果: {results_filter}")
except Exception as e:
    print(f"エラー: {e}")
    results_filter = None

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

ScreenerFilter使用結果: BacktestResults(trades=186, win_rate=55.4%, profit_factor=1.32)


In [27]:
# パラメータ比較: 損切りラインの最適化
stop_loss_levels = [-0.05, -0.08, -0.10, -0.12, -0.15]
comparison_results = []

for sl in stop_loss_levels:
    bt_test = Backtester(cash=1_000_000)
    bt_test.add_signal("golden_cross", short=5, long=25)
    bt_test.add_exit_rule("stop_loss", threshold=sl)
    bt_test.add_exit_rule("take_profit", threshold=0.20)

    try:
        res = bt_test.run(symbols=["7203"], start="2024-01-01", end="2024-12-31")
        summary = res.summary()
        comparison_results.append(
            {
                "stop_loss": sl,
                "win_rate": summary["win_rate"],
                "avg_return": summary["avg_return"],
                "profit_factor": summary["profit_factor"],
                "total_trades": summary["total_trades"],
            }
        )
    except Exception as e:
        print(f"SL={sl}: エラー - {e}")

if comparison_results:
    comparison_df = pd.DataFrame(comparison_results)
    print("=== 損切りライン比較 ===")
    display(comparison_df)

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

=== 損切りライン比較 ===


,stop_loss,win_rate,avg_return,profit_factor,total_trades
0,-0.05,0.2000,-0.0001,0.69,5
1,-0.08,0.2500,-0.0003,0.48,4
2,-0.10,0.2500,-0.0004,0.40,4
3,-0.12,0.3333,-0.0006,0.38,3
4,-0.15,0.3333,-0.0007,0.34,3


In [28]:
# シグナル別パフォーマンス比較
signal_configs = [
    {"name": "golden_cross", "params": {"short": 5, "long": 25}},
    {"name": "rsi_oversold", "params": {"threshold": 30}},
    {"name": "macd_cross", "params": {"fast": 12, "slow": 26, "signal_period": 9}},
    {
        "name": "bollinger_breakout",
        "params": {"period": 20, "std_dev": 2.0, "direction": "up"},
    },
]

signal_results = []

for config in signal_configs:
    bt_sig = Backtester(cash=1_000_000)
    bt_sig.add_signal(config["name"], **config["params"])
    bt_sig.add_exit_rule("stop_loss", threshold=-0.10)
    bt_sig.add_exit_rule("take_profit", threshold=0.20)

    try:
        res = bt_sig.run(symbols=["7203"], start="2024-01-01", end="2024-12-31")
        summary = res.summary()
        signal_results.append(
            {
                "signal": config["name"],
                "trades": summary["total_trades"],
                "win_rate": summary["win_rate"],
                "avg_return": summary["avg_return"],
                "sharpe": summary["sharpe_ratio"],
            }
        )
    except Exception as e:
        print(f"{config['name']}: エラー - {e}")

if signal_results:
    signal_df = pd.DataFrame(signal_results)
    print("=== シグナル別パフォーマンス ===")
    display(signal_df)

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

=== シグナル別パフォーマンス ===


,signal,trades,win_rate,avg_return,sharpe
0,golden_cross,4,0.2500,-0.0004,-0.77
1,rsi_oversold,2,0.5000,0.0002,0.17
2,macd_cross,6,0.3333,0.0000,-0.08
3,bollinger_breakout,3,0.6667,0.0008,1.33


---
## 6. VirtualPortfolio: 仮想ポートフォリオ

`VirtualPortfolio`クラスは、仮想的なポートフォリオを作成・追跡します。

### 主な機能
- 銘柄の購入/売却
- ポートフォリオ価値の追跡
- JSON形式での永続化（`data/portfolios/`）
- スクリーナー結果からの一括購入

In [29]:
# VirtualPortfolioの初期化
# 名前を指定して作成（既存があれば読み込み）
vp = VirtualPortfolio("demo_portfolio_2025")
print(f"ポートフォリオ: {vp}")

ポートフォリオ: VirtualPortfolio(name='demo_portfolio_2025', holdings=1)


In [30]:
# 銘柄の購入: 株数指定
try:
    vp.buy("7203", shares=100, price=2500)  # 価格を明示的に指定
    print("トヨタ(7203) 100株 @2,500円 購入完了")
except Exception as e:
    print(f"購入エラー: {e}")

トヨタ(7203) 100株 @2,500円 購入完了


In [31]:
# 銘柄の購入: 金額指定
try:
    vp.buy("9984", amount=500000)  # 50万円分購入（現在価格で株数計算）
    print("ソフトバンクG(9984) 50万円分 購入完了")
except Exception as e:
    print(f"購入エラー: {e}")

ソフトバンクG(9984) 50万円分 購入完了


In [32]:
# summary(): ポートフォリオサマリー
summary = vp.summary()
print("=== ポートフォリオサマリー ===")
print(f"投資総額: ¥{summary['total_investment']:,.0f}")
print(f"評価額: ¥{summary['current_value']:,.0f}")
print(f"損益: ¥{summary['total_pnl']:,.0f}")
print(f"リターン: {summary['return_pct']:.2%}")

=== ポートフォリオサマリー ===
投資総額: ¥2,734,120
評価額: ¥2,854,720
損益: ¥120,600
リターン: 4.41%


In [33]:
# holdings(): 保有銘柄一覧
holdings = vp.holdings()
print("=== 保有銘柄 ===")
display(holdings)

=== 保有銘柄 ===


,symbol,shares,avg_price,current_price,pnl
0,9984,635,3912.0,3912.0,0.0
1,7203,100,2500.0,3706.0,120600.0


In [34]:
# performance(): パフォーマンス推移
perf = vp.performance(days=30)
if not perf.empty:
    print(f"過去30日間のパフォーマンスデータ: {len(perf)}日分")
    display(perf.tail())
else:
    print("パフォーマンスデータなし")

過去30日間のパフォーマンスデータ: 21日分


,date,portfolio_value
16,2026-01-30,3051055.0
17,2026-02-02,2950650.0
18,2026-02-03,3089900.0
19,2026-02-04,3043945.0
20,2026-02-05,2854720.0


In [35]:
# plot(): ポートフォリオチャート
fig = vp.plot()
fig.show()

In [36]:
# 売却: 一部売却
try:
    vp.sell("7203", shares=50)  # 50株売却
    print("トヨタ(7203) 50株 売却完了")
except Exception as e:
    print(f"売却エラー: {e}")

トヨタ(7203) 50株 売却完了


In [37]:
# 売却: 全売却
try:
    vp.sell_all("7203")  # 残り全部売却
    print("トヨタ(7203) 全株売却完了")
except Exception as e:
    print(f"売却エラー: {e}")

トヨタ(7203) 全株売却完了


In [38]:
# buy_from_screener(): スクリーナー結果から一括購入
vp_screener = VirtualPortfolio("screener_portfolio_2025")

try:
    vp_screener.buy_from_screener(
        screener_filter={"composite_score_min": 80, "hl_ratio_min": 85},
        amount_per_stock=100000,  # 各銘柄10万円
        max_stocks=5,
    )
    print(f"スクリーナーから購入完了: {vp_screener}")
    display(vp_screener.holdings())
except Exception as e:
    print(f"購入エラー: {e}")

スクリーナーから購入完了: VirtualPortfolio(name='screener_portfolio_2025', holdings=5)


,symbol,shares,avg_price,current_price,pnl
0,77710,1010,495.0,495.0,0.0
1,39020,295,1687.0,1687.0,0.0
2,83860,50,9440.0,9440.0,0.0
3,69410,60,7730.0,7730.0,0.0
4,93670,195,2507.0,2507.0,0.0


In [39]:
# ScreenerFilterオブジェクトを使用した購入
growth_filter = ScreenerFilter(
    composite_score_min=75.0, hl_ratio_min=80.0, roe_min=15.0, limit=10
)

vp_growth = VirtualPortfolio("growth_portfolio_2025")

try:
    vp_growth.buy_from_screener(screener_filter=growth_filter, amount_per_stock=200000)
    print(f"成長株ポートフォリオ: {vp_growth}")
    print(vp_growth.summary())
except Exception as e:
    print(f"購入エラー: {e}")

成長株ポートフォリオ: VirtualPortfolio(name='growth_portfolio_2025', holdings=10)
{'total_investment': 9813750.0, 'current_value': 9813750.0, 'total_pnl': 0.0, 'return_pct': 0.0}


---
## 7. 実践ワークフロー: スクリーニングから投資判断まで

実際の投資分析では、以下のワークフローが効果的です：

1. **Step1**: スクリーニングで候補銘柄を抽出
2. **Step2**: バックテストで戦略を検証
3. **Step3**: 結果を詳細分析
4. **Step4**: 仮想ポートフォリオに組み入れ
5. **Step5**: チャート分析で最終確認

In [40]:
# Step1: スクリーニングで候補抽出
print("=== Step1: スクリーニング ===")

screener = StockScreener()

# 高スコア + 上昇トレンド + 過熱感なし
candidates = screener.filter(
    composite_score_min=75.0, hl_ratio_min=80.0, rsi_max=70.0, limit=10
)

print(f"候補銘柄数: {len(candidates)}")
if not candidates.empty:
    display(
        candidates[
            ["Code", "composite_score", "HlRatio", "RelativeStrengthIndex"]
        ].head()
    )

=== Step1: スクリーニング ===
候補銘柄数: 10


,Code,composite_score,HlRatio,RelativeStrengthIndex
0,92870,85.225983,99.630996,69.683960
1,28090,85.103046,100.000000,69.007615
2,59220,84.890474,98.652291,69.823893
3,97190,84.331253,99.329917,67.748215
4,80140,84.111881,96.542553,69.987149


In [41]:
# Step2: バックテストで戦略検証
print("=== Step2: バックテスト ===")

if not candidates.empty:
    symbols = candidates["Code"].tolist()[:5]  # 上位5銘柄

    bt = Backtester(cash=1_000_000)
    bt.add_signal("golden_cross", short=5, long=25)
    bt.add_signal("rsi_oversold", threshold=30)
    bt.add_exit_rule("stop_loss", threshold=-0.10)
    bt.add_exit_rule("take_profit", threshold=0.20)
    bt.add_exit_rule("trailing_stop", threshold=-0.05)

    try:
        results = bt.run(symbols=symbols, start="2024-01-01", end="2024-12-31")
        print(f"バックテスト完了: {results}")
    except Exception as e:
        print(f"バックテストエラー: {e}")
        results = None
else:
    print("候補銘柄がありません")
    results = None

=== Step2: バックテスト ===


Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/236 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

バックテスト完了: BacktestResults(trades=24, win_rate=33.3%, profit_factor=0.62)


In [42]:
# Step3: 結果の詳細分析
print("=== Step3: 結果分析 ===")

if results:
    summary = results.summary()

    print("\n【パフォーマンスサマリー】")
    print(f"  総取引: {summary['total_trades']}回")
    print(f"  勝率: {summary['win_rate']:.1%}")
    print(f"  プロフィットファクター: {summary['profit_factor']:.2f}")
    print(f"  シャープレシオ: {summary['sharpe_ratio']:.2f}")

    # 銘柄別分析
    by_symbol = results.by_symbol()
    if not by_symbol.empty:
        print("\n【銘柄別パフォーマンス】")
        display(by_symbol)

    # チャート表示
    fig = results.plot()
    fig.show()
else:
    print("分析する結果がありません")

=== Step3: 結果分析 ===

【パフォーマンスサマリー】
  総取引: 24回
  勝率: 33.3%
  プロフィットファクター: 0.62
  シャープレシオ: -0.75

【銘柄別パフォーマンス】


,symbol,trades,avg_return,total_pnl,win_rate
0,28090,3,0.000311,72259.0,0.333333
4,97190,7,0.000075,41416.0,0.571429
2,80140,6,-0.000089,-65645.0,0.333333
1,59220,3,-0.000493,-143040.0,0.333333
3,92870,5,-0.000554,-244100.0,0.000000


In [43]:
# Step4: 仮想ポートフォリオへの組み入れ
print("=== Step4: ポートフォリオ構築 ===")

if results:
    # 好成績の銘柄を特定
    by_symbol = results.by_symbol()
    profitable = by_symbol[by_symbol["total_pnl"] > 0]

    if not profitable.empty:
        vp_final = VirtualPortfolio("strategy_portfolio_2025")

        for _, row in profitable.iterrows():
            symbol = row["symbol"]
            try:
                vp_final.buy(symbol, amount=200000)  # 各銘柄20万円
                print(f"  {symbol}: 20万円購入完了")
            except Exception as e:
                print(f"  {symbol}: 購入失敗 - {e}")

        print(f"\nポートフォリオ構築完了: {vp_final}")
        print(vp_final.summary())
    else:
        print("利益の出た銘柄がありません")
else:
    print("バックテスト結果がありません")

=== Step4: ポートフォリオ構築 ===
  28090: 20万円購入完了
  97190: 20万円購入完了

ポートフォリオ構築完了: VirtualPortfolio(name='strategy_portfolio_2025', holdings=2)
{'total_investment': 1779460.0, 'current_value': 1779460.0, 'total_pnl': 0.0, 'return_pct': 0.0}


In [44]:
# Step5: TechnicalAnalyzerとの連携チャート分析
print("=== Step5: チャート分析 ===")

analyzer = TechnicalAnalyzer(source="jquants")

if not candidates.empty:
    top_code = candidates.iloc[0]["Code"]
    print(f"分析対象: {top_code}")

    # フル機能チャート
    fig = analyzer.plot_chart(
        top_code,
        show_sma=[5, 25, 75],
        show_bb=True,
        show_rsi=True,
        show_macd=True,
        show_signals=True,
        signal_patterns=[(5, 25)],
        start="2024-01-01",
    )
    fig.update_layout(title=f"{top_code} テクニカル分析")
    fig.show()

    # 既存分析結果も確認
    existing = analyzer.load_existing_analysis(top_code)
    if existing["minervini"]:
        print("\nMinervini分析結果:")
        for key in ["Type_1", "Type_2", "Type_3", "Type_4"]:
            if key in existing["minervini"]:
                print(f"  {key}: {existing['minervini'][key]}")
else:
    print("候補銘柄がありません")

=== Step5: チャート分析 ===
分析対象: 92870



Minervini分析結果:
  Type_1: 1.0
  Type_2: 1.0
  Type_3: 1.0
  Type_4: 1.0


---
## 8. 高度な使い方

### 複数戦略の比較

In [45]:
# 複数戦略の定義
strategies = {
    "ゴールデンクロス(5/25)": {
        "signals": [("golden_cross", {"short": 5, "long": 25})],
        "exits": {"stop_loss": -0.10, "take_profit": 0.20},
    },
    "RSI逆張り": {
        "signals": [("rsi_oversold", {"threshold": 30})],
        "exits": {"stop_loss": -0.08, "take_profit": 0.12},
    },
    "MACD+トレーリング": {
        "signals": [("macd_cross", {"fast": 12, "slow": 26, "signal_period": 9})],
        "exits": {"trailing_stop": -0.05, "max_holding_days": 30},
    },
    "複合シグナル": {
        "signals": [
            ("golden_cross", {"short": 5, "long": 25}),
            ("rsi_oversold", {"threshold": 30}),
        ],
        "exits": {"stop_loss": -0.10, "take_profit": 0.20, "trailing_stop": -0.05},
    },
}

print(f"定義した戦略数: {len(strategies)}")

定義した戦略数: 4


In [46]:
# 戦略比較の実行
strategy_results = []

for name, config in strategies.items():
    bt = Backtester(cash=1_000_000)

    # シグナル追加
    for signal_name, params in config["signals"]:
        bt.add_signal(signal_name, **params)

    # 退出ルール追加
    for rule_name, value in config["exits"].items():
        if rule_name == "max_holding_days":
            bt.add_exit_rule(rule_name, days=value)
        else:
            bt.add_exit_rule(rule_name, threshold=value)

    try:
        results = bt.run(
            symbols=["7203", "9984", "6758"], start="2024-01-01", end="2024-12-31"
        )
        summary = results.summary()
        strategy_results.append(
            {
                "戦略": name,
                "取引数": summary["total_trades"],
                "勝率": f"{summary['win_rate']:.1%}",
                "平均リターン": f"{summary['avg_return']:.2%}",
                "PF": summary["profit_factor"],
                "シャープ": summary["sharpe_ratio"],
            }
        )
    except Exception as e:
        print(f"{name}: エラー - {e}")

if strategy_results:
    comparison_df = pd.DataFrame(strategy_results)
    print("=== 戦略比較 ===")
    display(comparison_df)

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

=== 戦略比較 ===


,戦略,取引数,勝率,平均リターン,PF,シャープ
0,ゴールデンクロス(5/25),12,33.3%,-0.07%,0.31,-0.86
1,RSI逆張り,7,71.4%,0.06%,2.10,1.17
2,MACD+トレーリング,24,41.7%,0.00%,0.92,0.00
3,複合シグナル,24,41.7%,-0.00%,0.89,-0.13


In [47]:
# セクター別分析（セクターマップが必要）
# by_sector()メソッドを使用

sector_map = {
    "7203": "自動車",
    "9984": "情報通信",
    "6758": "電気機器",
    "6501": "電気機器",
    "8035": "電気機器",
}

if results_combo:
    sector_analysis = results_combo.by_sector(sector_map)
    if not sector_analysis.empty:
        print("=== セクター別パフォーマンス ===")
        display(sector_analysis)

=== セクター別パフォーマンス ===


,sector,trades,avg_return,total_pnl,win_rate
2,電気機器,9,0.000140,107210.0,0.555556
1,自動車,10,0.000096,45548.0,0.400000
0,情報通信,8,0.000036,-22753.0,0.375000


---
## 9. Tips & ベストプラクティス

### Backtester
- 複数シグナルはOR条件（どれか1つで発動）
- `max_workers`で並列処理を制御（デフォルト: min(銘柄数, 4)）
- 損切りは必ず負の値（例: `-0.10`）、利確は正の値（例: `0.20`）

### BacktestResults
- `summary()`で主要指標を確認
- `by_symbol()`で銘柄ごとの貢献度を分析
- `export()`でExcel出力すると複数シート（Summary, Trades, By Symbol, Monthly）

### VirtualPortfolio
- `data/portfolios/`にJSONで永続化される
- 同じ名前で初期化すると既存データを読み込む
- `buy_from_screener()`でスクリーナー結果を一括購入

### 一般的な注意事項
- 過去のパフォーマンスは将来を保証しない
- 手数料・スリッページを考慮する
- 複数期間でバックテストして堅牢性を確認

In [48]:
# 堅牢性テスト: 異なる期間でのバックテスト
periods = [
    ("2023-01-01", "2023-12-31"),
    ("2024-01-01", "2024-12-31"),
    ("2023-01-01", "2024-12-31"),
]

robustness_results = []

for start, end in periods:
    bt = Backtester(cash=1_000_000)
    bt.add_signal("golden_cross", short=5, long=25)
    bt.add_exit_rule("stop_loss", threshold=-0.10)
    bt.add_exit_rule("take_profit", threshold=0.20)

    try:
        res = bt.run(symbols=["7203"], start=start, end=end)
        summary = res.summary()
        robustness_results.append(
            {
                "期間": f"{start} ~ {end}",
                "取引数": summary["total_trades"],
                "勝率": summary["win_rate"],
                "PF": summary["profit_factor"],
            }
        )
    except Exception as e:
        print(f"{start}~{end}: エラー - {e}")

if robustness_results:
    robust_df = pd.DataFrame(robustness_results)
    print("=== 堅牢性テスト ===")
    display(robust_df)

Backtest.run:   0%|          | 0/245 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/490 [00:00<?, ?bar/s]

=== 堅牢性テスト ===


,期間,取引数,勝率,PF
0,2023-01-01 ~ 2023-12-31,3,0.6667,3.45
1,2024-01-01 ~ 2024-12-31,4,0.2500,0.40
2,2023-01-01 ~ 2024-12-31,7,0.5714,1.90


---
## 10. まとめ

このノートブックでは、バックテストと仮想ポートフォリオ機能を使った投資戦略検証ワークフローを説明しました。

### 概要

1. **Backtester**
   - 9種類のテクニカルシグナル対応
   - 4種類の退出ルール
   - 並列バックテスト実行
   - スクリーナー連携

2. **BacktestResults**
   - パフォーマンス指標の計算
   - 銘柄別・月次・年次分析
   - Plotlyによる可視化
   - CSV/Excel/HTMLエクスポート

3. **VirtualPortfolio**
   - 仮想ポートフォリオの作成・追跡
   - JSON永続化
   - スクリーナー連携購入
   - パフォーマンスチャート

### 次のステップ
- 独自のシグナル条件でバックテストを実行
- 複数戦略を比較して最適な組み合わせを発見
- 仮想ポートフォリオで長期運用をシミュレーション